<a href="https://colab.research.google.com/github/Saibhavyabhavi/FMML_Project_and_Labs/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [18]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

AttributeError: module 'numpy' has no attribute 'int'.
`np.int` was a deprecated alias for the builtin `int`. To avoid this error in existing code, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

Here is a function for calculating the 1-nearest neighbours

In [4]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [5]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [6]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [7]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [8]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [9]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [10]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.0003234152652005175


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [12]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.009205426356589148
Validation accuracy using random classifier is  0.0


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [13]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.009748964172556666


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [14]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.009893822393822393


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

1A)How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?

Increase Percentage of Validation Set:

Positive Impact on Validation Accuracy: When you allocate a larger percentage of your data to the validation set, you are effectively using more data to evaluate your model's performance. This can lead to a more accurate estimate of your model's performance on unseen data, resulting in a higher validation accuracy.

Reduced Training Data: One trade-off is that you'll have less data available for training your model. With a smaller training set, your model may not learn as effectively from the data, and it might be prone to overfitting, especially if the dataset is small to begin with.

Risk of Underfitting: If the validation set percentage is very high, you might end up with too little data for training, potentially causing your model to underfit the training data.

Reduce Percentage of Validation Set:

Positive Impact on Training Accuracy: Allocating a smaller percentage to the validation set means you have more data available for training. This can lead to a model that learns more effectively from the training data and achieves higher training accuracy.

Lower Validation Accuracy: On the downside, a smaller validation set may result in a less reliable estimate of your model's performance. With less data for validation, your estimate may be more susceptible to random variations in the validation set, potentially leading to a lower validation accuracy.

Risk of Overfitting: With a smaller validation set, your model might appear to perform well on the validation set even if it is overfitting the data. This can result in an optimistic but unrealistic estimate of model performance.

2A)How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?

Training Set Size:

Larger Training Set: A larger training set typically results in a better-trained model because it has more data to learn from. When your model is well-trained, it has a higher chance of generalizing well to unseen data, including the test set. This, in turn, can lead to a more accurate prediction of test set performance using the validation set.

Smaller Training Set: Conversely, with a smaller training set, your model may not learn as effectively, potentially leading to overfitting. Overfit models tend to perform well on the training data but poorly on unseen data (test set). In such cases, the validation set's accuracy may not be a good predictor of test set accuracy.

Validation Set Size:

Larger Validation Set: A larger validation set can provide a more reliable estimate of your model's performance because it's based on more data. If you have a substantial validation set, it's more likely to reflect the true generalization performance of your model. As a result, the validation set's accuracy is a better predictor of the test set accuracy.

Smaller Validation Set: A smaller validation set can still provide valuable insights into model performance, but it may be more sensitive to variations in the data. If the validation set is too small, it might not capture the full range of scenarios and patterns present in the test set, which could lead to less accurate predictions of test set accuracy.

Generalization: Ultimately, the goal is to ensure that your model generalizes well to unseen data. The accuracy on the validation set is an indicator of this generalization ability. A model that performs well on the validation set is more likely to perform well on the test set, but this relationship becomes stronger when you have both a sufficiently large training set and a sufficiently large validation set.

3A)What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

The reasons for reserving around 20-30% of your dataset for the validation set are:

Balanced Trade-off: It provides a reasonable balance between having enough data for training the model effectively (70-80%) and having a sufficiently large validation set (20-30%) to estimate model performance accurately.

Common Practice: It is a common and widely accepted practice in the machine learning community to use this split ratio, making it a good starting point.

Detecting Overfitting: A sizable validation set helps in detecting overfitting and assessing how well the model generalizes to unseen data.

Practicality: It's practical and often works well across various datasets and model complexities.

Remember that the choice of the validation set percentage can vary based on specific circumstances, but 20-30% is a reasonable guideline in many cases.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [15]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [16]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.008690325202174656
test accuracy is  0.009893822393822393


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


1A)Does averaging the validation accuracy across multiple splits give more consistent results?

Yes, averaging the validation accuracy across multiple splits of your dataset can indeed provide more consistent and reliable results compared to relying on a single split. This technique is commonly known as "cross-validation," and it helps to reduce the impact of data randomness and variability in your model's performance evaluation.

Here's why averaging validation accuracy across multiple splits is beneficial:

Reduced Overfitting: Averaging results over multiple splits helps to mitigate the risk of overfitting to a specific dataset split. If you only evaluate your model on a single validation set, it might perform exceptionally well or poorly due to the luck of the split, leading to an inaccurate representation of its actual performance.

Robustness to Data Variability: Datasets can have inherent variability, such as variations in data distribution or the presence of outliers. By using multiple splits, you get a better sense of how well your model generalizes to different subsets of the data.

2A)Does it give more accurate estimate of test accuracy?

Averaging the validation accuracy across multiple splits in cross-validation does not directly provide a more accurate estimate of the test accuracy. Instead, it provides a more accurate estimate of how well your model is likely to perform on unseen data that is similar to your training data. This estimate is often referred to as the "cross-validated performance" or "cross-validation"

While cross-validation can give you a good estimate of your model's generalization performance, it doesn't directly provide the test accuracy. However, if your cross-validated performance is consistent and high across multiple splits, it can provide some confidence that your model is likely to perform well on the test data, assuming that the test data is representative of the same distribution as your training data.

In practice, the test dataset remains crucial for confirming your model's performance before deploying it in real-world applications, as it serves as the ultimate benchmark of its accuracy on new, independent samples. Cross-validation is a valuable tool for model selection, hyperparameter tuning, and gaining insights into your model's behavior, but it does not replace the need for a separate test dataset for final evaluation.

3A)What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?

The number of iterations or folds in cross-validation can have an effect on the estimate of your model's performance. The impact of the number of iterations on the estimate depends on several factors:

Bias-Variance Trade-Off: There is a trade-off between bias and variance when choosing the number of iterations (folds) in cross-validation. With a higher number of folds (e.g., k-fold cross-validation with a large k), you reduce the bias in your estimate because you're using more data for both training and validation. However, this can lead to higher variance in your estimate because each fold represents a smaller portion of your data, which can make the estimate more sensitive to random fluctuations.

Computational Cost: Increasing the number of iterations in cross-validation can significantly increase the computational cost, especially for large datasets and complex models. Each iteration involves training and evaluating the model, and more iterations mean more training and evaluation steps.

Data Availability: The choice of the number of iterations also depends on the amount of data you have available. If you have a small dataset, using a very high number of folds might not be feasible, and you could end up with very small training and validation sets in each fold.

In general, there isn't a one-size-fits-all answer to whether a higher number of iterations leads to a better estimate. The choice of the number of iterations (e.g., k in k-fold cross-validation) should be made based on the specific characteristics of your dataset and the goals of your analysis:

Standard Practice: 5-fold or 10-fold cross-validation is commonly used as a reasonable trade-off between bias and variance. These values are often a good starting point.

Leave-One-Out Cross-Validation (LOOCV): In LOOCV, you use one data point as the validation set in each iteration. It provides an unbiased estimate but can have high variance and be computationally expensive. It's useful when you have a very small dataset.

Stratified Cross-Validation: In cases of imbalanced datasets, you may want to use stratified cross-validation to ensure that each fold maintains the class distribution of the original dataset.

4A)Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?

Increasing the number of iterations (folds) in cross-validation can help mitigate the impact of having a very small training dataset or validation dataset to some extent, but it may not fully address the limitations associated with small datasets. Here are some considerations:

Advantages of Increasing Iterations with small datasets:

More Reliable Performance Estimate: Increasing iterations can provide a more reliable estimate of your model's performance compared to relying on a single split, especially when your dataset is too small to provide robust performance metrics.

Limitations of Increasing Iterations with Small Datasets:

Increased Computational Cost: While increasing iterations can improve the reliability of your estimates, it also increases the computational cost. Training and evaluating your model multiple times with small datasets can be computationally expensive, especially for complex models.